In [12]:
!pip install pygame ipympl

In [13]:
%matplotlib widget

In [14]:
pip install gym numpy tkinter

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tkinter (from versions: none)
ERROR: No matching distribution found for tkinter


In [11]:
pip install gym

Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install numpy tkinter

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tkinter (from versions: none)
ERROR: No matching distribution found for tkinter


In [17]:
import tkinter as tk
root = tk.Tk()
root.title("Test")
root.mainloop()

In [16]:
!pip install numpy pillow

In [18]:
# Missile 1
import tkinter as tk
import numpy as np
import random
from collections import defaultdict
import pickle
import os

# ========================
# 1. Game Environment Module (GameEnv)
# ========================
class GameEnv:
    def __init__(self, grid_width=400, grid_height=600):
        self.grid_width = grid_width
        self.grid_height = grid_height
        
        # Game parameters
        self.player_size = 20
        self.obstacle_width = 50
        self.gap_size = 150  # Increase gap to lower initial difficulty
        self.player_speed = 5
        self.obstacle_speed = 3
        
        # Reinforcement learning parameters
        self.action_space = 2  # 0: Don't move, 1: Jump
        self.observation_space = (3,)  # [Player Y, recent obstacle X, obstacle top Y]
        self.reset()

    def reset(self):
        """Reset Game State"""
        self.player_y = self.grid_height // 2
        self.velocity = 0
        self.obstacles = [{
            'x': self.grid_width,
            'gap_y': random.randint(100, self.grid_height - 100 - self.gap_size)
        }]
        self.score = 0
        self.done = False
        return self._get_state()

    def _get_state(self):
        """Obtain simplified state"""
        nearest = self.obstacles[0]
        return np.array([
            self.player_y // 50,  # Discretize player position
            nearest['x'] // 50,   # Discretize obstacle position
            nearest['gap_y'] // 50  # Discretize obstacle gap position
        ], dtype=int)

    def step(self, action):
        """Execute an action and return to a new state"""
        # Handling jumping actions
        if action == 1:
            self.velocity = -8
        
        # Physical simulation
        self.velocity += 0.5
        self.player_y = np.clip(
            self.player_y + self.velocity,
            0, self.grid_height - self.player_size
        )
        
        # Moving obstacles
        for obs in self.obstacles:
            obs['x'] -= self.obstacle_speed
        
        # Generate new obstacles (reduce generation interval)
        if self.obstacles[-1]['x'] < self.grid_width - 250:
            self.obstacles.append({
                'x': self.grid_width,
                'gap_y': random.randint(100, self.grid_height - 100 - self.gap_size)
            })
        
        # Remove obstacles outside the screen
        if self.obstacles[0]['x'] < -self.obstacle_width:
            self.obstacles.pop(0)
        
        # Collision detection
        reward = 0.5  # Basic survival reward
        nearest = self.obstacles[0]
        
        # Calculate the player center position
        player_center = self.player_y + self.player_size / 2
        gap_center = nearest['gap_y'] + self.gap_size / 2
        
        # Direction reward: Encourage approaching the center of the obstacle gap
        distance_to_center = abs(player_center - gap_center)
        reward += 1.0 - (distance_to_center / (self.gap_size / 2))  # The closer you are, the higher the reward
        
        # Accurate collision detection
        collision_vertical = (
            self.player_y < nearest['gap_y'] or
            self.player_y + self.player_size > nearest['gap_y'] + self.gap_size
        )
        collision_horizontal = (
            nearest['x'] < 50 + self.player_size and  # Player X's position is fixed at 50
            nearest['x'] + self.obstacle_width > 50
        )
        
        if collision_vertical and collision_horizontal:
            self.done = True
            reward = -10  # Collision penalty
            print(f"Collision! Player Y: {self.player_y}, Gap Y: {nearest['gap_y']}")
        
        # Score reward
        if nearest['x'] + self.obstacle_width < 50:  # Player X position fixed at 50
            self.score += 1
            reward += 5  # Significant score reward
            print(f"Score: {self.score}")
        
        return self._get_state(), reward, self.done, {}

# ========================
# 2. Q-learning Agent Module (Improved Version)
# ========================
class QLearningAgent:
    def __init__(self, state_dims, action_dims, 
                 lr=0.3, gamma=0.95, epsilon=1.0, epsilon_min=0.01, epsilon_decay=0.997):
        self.q_table = defaultdict(lambda: np.zeros(action_dims))
        self.lr = lr
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.state_dims = state_dims
        self.action_dims = action_dims

    def get_state_key(self, state):
        """Discretize continuous state into 10 intervals"""
        return tuple(state)
    
    def choose_action(self, state):
        """Epsilon-greedy strategy with decay"""
        if random.random() < self.epsilon:
            action = random.randint(0, self.action_dims - 1)
        else:
            state_key = self.get_state_key(state)
            action = np.argmax(self.q_table[state_key])
        
        # Decay exploration rate
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)
        return action
    
    def learn(self, state, action, reward, next_state):
        """Q-learning update (with reward clipping)"""
        state_key = self.get_state_key(state)
        next_key = self.get_state_key(next_state)
        
        current_q = self.q_table[state_key][action]
        max_next_q = np.max(self.q_table[next_key])
        
        # Limit reward range to prevent Q-value explosion
        clipped_reward = np.clip(reward, -10, 10)
        
        new_q = (1 - self.lr) * current_q + self.lr * (clipped_reward + self.gamma * max_next_q)
        self.q_table[state_key][action] = new_q

    def save_model(self, filename="q_agent.pkl"):
        """Save Q-table to file"""
        with open(filename, "wb") as f:
            pickle.dump(dict(self.q_table), f)  # Convert defaultdict to regular dict for saving
        print(f"Model saved to {filename}")

    def load_model(self, filename="q_agent.pkl"):
        """Load Q-table from file"""
        try:
            with open(filename, "rb") as f:
                self.q_table = defaultdict(lambda: np.zeros(self.action_dims), pickle.load(f))
            print(f"Loaded model from {filename}")
        except FileNotFoundError:
            print("No saved model found, starting fresh.")

# ========================
# 3. Game Window Module
# ========================
class GameWindow:
    def __init__(self, root, env, agent, update_interval=50):
        self.root = root
        self.env = env
        self.agent = agent
        self.update_interval = update_interval
        
        # Create main interface
        self.create_ui()
        self.reset()

    def create_ui(self):
        """Create user interface"""
        # Game canvas
        self.canvas = tk.Canvas(self.root, width=self.env.grid_width, height=self.env.grid_height, bg="black")
        self.canvas.pack(side=tk.LEFT, padx=10, pady=10)
        
        # Information panel
        info_frame = tk.Frame(self.root)
        info_frame.pack(side=tk.RIGHT, padx=10, pady=10)
        
        # Current state
        self.state_label = tk.Label(info_frame, text="State: ", font=("Arial", 12))
        self.state_label.pack(pady=5)
        
        # Agent action
        self.action_label = tk.Label(info_frame, text="Action: ", font=("Arial", 12))
        self.action_label.pack(pady=5)
        
        # Reward/Penalty
        self.reward_label = tk.Label(info_frame, text="Reward: ", font=("Arial", 12))
        self.reward_label.pack(pady=5)
        
        # Score
        self.score_label = tk.Label(info_frame, text="Score: 0", font=("Arial", 12))
        self.score_label.pack(pady=5)
        
        # Reset button
        reset_button = tk.Button(info_frame, text="Reset", command=self.reset, font=("Arial", 12))
        reset_button.pack(pady=10)
    
    def reset(self):
        """Reset game screen"""
        self.canvas.delete("all")
        self.state = self.env.reset()
        self.done = False
        self.update()
    
    def draw_objects(self):
        """Draw game elements"""
        # Draw player
        x = 50
        y = self.env.player_y
        self.player = self.canvas.create_rectangle(
            x, y, x + self.env.player_size, y + self.env.player_size,
            fill="blue", outline=""
        )
        
        # Draw obstacles
        for obs in self.env.obstacles:
            # Top obstacle
            self.canvas.create_rectangle(
                obs['x'], 0,
                obs['x'] + self.env.obstacle_width, obs['gap_y'],
                fill="red", outline=""
            )
            # Bottom obstacle
            self.canvas.create_rectangle(
                obs['x'], obs['gap_y'] + self.env.gap_size,
                obs['x'] + self.env.obstacle_width, self.env.grid_height,
                fill="red", outline=""
            )
        
        # Update score
        self.score_label.config(text=f"Score: {self.env.score}")
    
    def update(self):
        """Update game screen"""
        if not self.done:
            # Agent decision
            action = self.agent.choose_action(self.state)
            next_state, reward, done, _ = self.env.step(action)
            
            # Learning update
            self.agent.learn(self.state, action, reward, next_state)
            
            # Refresh screen
            self.canvas.delete("all")
            self.draw_objects()
            self.state = next_state
            self.done = done
            
            # Update information panel
            self.state_label.config(text=f"State: {self.state}")
            self.action_label.config(text=f"Action: {action}")
            self.reward_label.config(text=f"Reward: {reward}")
            
            # Continue loop
            self.root.after(self.update_interval, self.update)
        else:
            # Show game over
            self.canvas.create_text(
                self.env.grid_width // 2, self.env.grid_height // 2,
                text=f"Game Over! Score: {self.env.score}",
                fill="white", font=("Arial", 20)
            )
            # Reset game (shorten wait time)
            self.root.after(1000, self.reset)

# ========================
# 4. Main Program
# ========================
if __name__ == "__main__":
    # Initialize components
    env = GameEnv()
    agent = QLearningAgent(
        state_dims=3,
        action_dims=2,
        lr=0.3,          # Increase learning rate
        gamma=0.95,      # Focus more on recent rewards
        epsilon=1.0,     # Initial high exploration rate
        epsilon_min=0.01,
        epsilon_decay=0.997
    )
    
    # Attempt to load existing model
    agent.load_model()
    
    # Create main window
    root = tk.Tk()
    root.title("AI Obstacle Game (Optimized)")
    game = GameWindow(root, env, agent, update_interval=50)
    
    # Start training
    root.mainloop()

No saved model found, starting fresh.
Collision! Player Y: 0.0, Gap Y: 288
Collision! Player Y: 0.0, Gap Y: 292
Collision! Player Y: 0.0, Gap Y: 317
Collision! Player Y: 0.0, Gap Y: 108
Collision! Player Y: 0.0, Gap Y: 106
Collision! Player Y: 0.0, Gap Y: 308
Collision! Player Y: 0.0, Gap Y: 134
Collision! Player Y: 0.0, Gap Y: 162
Collision! Player Y: 0.0, Gap Y: 101
Collision! Player Y: 0.0, Gap Y: 189
Collision! Player Y: 0.0, Gap Y: 271
Score: 1
Score: 2
Score: 3
Score: 4
Score: 5
Score: 6
Score: 7
Score: 8
Score: 9
Score: 10
Score: 11
Score: 12
Score: 13
Score: 14
Score: 15
Score: 16
Score: 17
Collision! Player Y: 0.0, Gap Y: 330


In [19]:
# Missile 2
import os
import tkinter as tk
import numpy as np
import random
from collections import defaultdict
from PIL import Image, ImageTk

# ========================
# User-defined path configuration
# ========================
IMAGE_PATHS = {
    "bullet": "bullet.jpg",
    "player": "player.jpg",
    "background": "background2.jpg",
    "enemy": "normal_obstacle.jpg"
}

# ========================
# Game constant configuration
# ========================
WIDTH, HEIGHT = 600, 800
PLAYER_SIZE = 50
BULLET_SIZE = 15
OBSTACLE_WIDTH = 80
GAP_SIZE = 220
BG_COLOR = "#1a1a1a"
UI_FONT = ("Segoe UI", 12)
ENEMY_SIZE = 40
BLINK_DURATION = 15
OBSTACLE_GAP = 300

# ========================
# Game resource loading
# ========================
def load_assets():
    assets = {}
    try:
        assets["player"] = ImageTk.PhotoImage(Image.open(IMAGE_PATHS["player"]).resize((PLAYER_SIZE, PLAYER_SIZE)))
        assets["bullet"] = ImageTk.PhotoImage(Image.open(IMAGE_PATHS["bullet"]).resize((BULLET_SIZE, BULLET_SIZE)))
        assets["background"] = ImageTk.PhotoImage(Image.open(IMAGE_PATHS["background"]).resize((WIDTH, HEIGHT)))
        assets["enemy"] = ImageTk.PhotoImage(Image.open(IMAGE_PATHS["enemy"]).resize((ENEMY_SIZE, ENEMY_SIZE)))
    except Exception as e:
        print(f"An error occurred while loading resources: {e}")
        exit(1)
    return assets

# ========================
# Core game logic (fixed version)
# ========================
class GameEnv:
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.player_y = HEIGHT // 2
        self.velocity = 0
        self.obstacles = []
        self.bullets = []
        self.enemies = []
        self.score = 0
        self.bullet_count = 10
        self.lives = 3
        self.done = False
        self.blink_counter = 0  # Scintillation counter
        self._generate_initial_obstacles()
    
    def _generate_initial_obstacles(self):
        for i in range(3):
            self.obstacles.append({
                "x": WIDTH + i * OBSTACLE_GAP,
                "gap_y": random.randint(150, HEIGHT - 150 - GAP_SIZE),
                "top_height": random.randint(150, HEIGHT - 150 - GAP_SIZE),
                "bottom_height": HEIGHT - (random.randint(150, HEIGHT - 150 - GAP_SIZE) + GAP_SIZE)
            })
    
    def _generate_obstacle(self):
        gap_y = random.randint(150, HEIGHT - 150 - GAP_SIZE)
        self.obstacles.append({
            "x": WIDTH,
            "gap_y": gap_y,
            "top_height": gap_y,
            "bottom_height": HEIGHT - (gap_y + GAP_SIZE)
        })
    
    def _generate_enemy(self):
        if random.random() < 0.02:  # 2% chance to generate enemies
            self.enemies.append({
                "x": WIDTH,
                "y": random.randint(100, HEIGHT - 100),
                "health": 1
            })
    
    def _get_state(self):
        if not self.obstacles:
            return np.zeros(10)
        
        nearest_obstacle = self.obstacles[0]
        nearest_enemy = min(self.enemies, key=lambda e: e["x"]) if self.enemies else None
        
        return np.array([
            self.player_y / HEIGHT,
            self.velocity / 20,
            (nearest_obstacle["x"] - 100) / WIDTH,
            nearest_obstacle["gap_y"] / HEIGHT,
            (nearest_obstacle["gap_y"] + GAP_SIZE) / HEIGHT,
            self.bullet_count / 20,
            nearest_enemy["x"] / WIDTH if nearest_enemy else 0,
            nearest_enemy["y"] / HEIGHT if nearest_enemy else 0,
            (self.player_y - nearest_obstacle["gap_y"]) / HEIGHT,
            (self.player_y - (nearest_obstacle["gap_y"] + GAP_SIZE)) / HEIGHT
        ])
    ###
    def step(self, action):
        # Handle player action
        if action == 1:  # jump
            self.velocity = -13
        elif action == 2 and self.bullet_count > 0:  # Launch missiles
            self.bullets.append({"x": 100, "y": self.player_y + PLAYER_SIZE//2})
            self.bullet_count -= 1

        # Physics simulation
        self.velocity += 0.65
        self.player_y = np.clip(self.player_y + self.velocity, 0, HEIGHT - PLAYER_SIZE)

        # Obstacle movement
        for obs in self.obstacles:
            obs["x"] -= 4

        # Bullet movement
        for bullet in self.bullets:
            bullet["x"] += 12

        # Enemy logic
        self._generate_enemy()
        for enemy in self.enemies[:]:
            enemy["x"] -= 4
            # Bullet and enemy collision detection
            for bullet in self.bullets[:]:
                if (enemy["x"] < bullet["x"] < enemy["x"] + ENEMY_SIZE and
                    enemy["y"] < bullet["y"] < enemy["y"] + ENEMY_SIZE):
                    self.enemies.remove(enemy)  # Enemy disappears after being hit
                    self.bullets.remove(bullet)
                    self.bullet_count += 5  # Reward for destroying
                    self.score += 10  # Score reward####

        # Obstacle collision detection (fixed version)
        reward = 0
        for obs in self.obstacles[:]:  # Iterate over all obstacles
            if (obs["x"] < 120 < obs["x"] + OBSTACLE_WIDTH and
                (self.player_y < obs["gap_y"] or 
                 self.player_y + PLAYER_SIZE > obs["gap_y"] + GAP_SIZE)):
                self.done = True
                reward = -25
                break  # Exit loop immediately after detecting collision

        # Remove obstacles outside of the screen
        self.obstacles = [obs for obs in self.obstacles if obs["x"] + OBSTACLE_WIDTH > 0]

        # Generate new obstacles
        if self.obstacles[-1]["x"] < WIDTH - OBSTACLE_GAP:
            self._generate_obstacle()

        # Enemy and player collision detection
        for enemy in self.enemies[:]:
            if (80 < enemy["x"] < 80 + PLAYER_SIZE and
                self.player_y < enemy["y"] + ENEMY_SIZE and
                self.player_y + PLAYER_SIZE > enemy["y"]):
                if self.blink_counter == 0:  # Only deduct life when not blinking
                    self.lives -= 1
                    self.blink_counter = BLINK_DURATION  # Trigger blinking
                    if self.lives <= 0:
                        self.done = True
                        reward = -30
                    else:
                        reward = -10
                    self.enemies.remove(enemy)  # Enemy disappears after collision

        # Update blinking state
        if self.blink_counter > 0:
            self.blink_counter -= 1

        return self._get_state(), reward, self.done

# ========================
# Q-Learning agent
# ========================
class QLearningAgent:
    def __init__(self):
        self.q_table = defaultdict(lambda: np.zeros(3))
        self.epsilon = 1.0
        self.lr = 0.8
        self.gamma = 0.98
    
    def choose_action(self, state):
        if random.random() < self.epsilon:
            return random.choice([0, 1, 2])
        return np.argmax(self.q_table[tuple(state)])
    
    def learn(self, state, action, reward, next_state):
        state_key = tuple(np.round(state, 2))
        next_key = tuple(np.round(next_state, 2))
        self.q_table[state_key][action] += self.lr * (
            reward + self.gamma * np.max(self.q_table[next_key]) - self.q_table[state_key][action]
        )
        self.epsilon = max(0.05, self.epsilon * 0.997)

# ========================
# Game window class (fixed version)
# ========================
class GameWindow:
    def __init__(self, root):
        self.assets = load_assets()
        self.env = GameEnv()
        self.agent = QLearningAgent()
        self.root = root
        self.canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg=BG_COLOR)
        self.canvas.pack()
        
        self.info_frame = tk.Frame(root, bg=BG_COLOR)
        self.info_frame.pack(pady=10)
        
        self.score_label = tk.Label(self.info_frame, text="Score: 0", font=UI_FONT, fg="white", bg=BG_COLOR)
        self.score_label.pack(side=tk.LEFT, padx=20)
        
        self.bullet_label = tk.Label(self.info_frame, text="Missiles: 10", font=UI_FONT, fg="#FF5555", bg=BG_COLOR)
        self.bullet_label.pack(side=tk.LEFT, padx=20)
        
        self.life_label = tk.Label(self.info_frame, text="Lives: ♥♥♥", font=UI_FONT, fg="#FF3366", bg=BG_COLOR)
        self.life_label.pack(side=tk.LEFT, padx=20)
        
        self.epsilon_label = tk.Label(self.info_frame, text="Explore: 100%", font=UI_FONT, fg="#55FF55", bg=BG_COLOR)
        self.epsilon_label.pack(side=tk.LEFT, padx=20)
        
        self.root.bind("<Up>", lambda e: self._handle_action(1))
        self.root.bind("<space>", lambda e: self._handle_action(2))
        
        self.update()

    def _handle_action(self, action):
        state, reward, done = self.env.step(action)
        if done:
            self._reset_game()

    def draw(self):
        self.canvas.delete("all")
        # Draw background
        self.canvas.create_image(0, 0, image=self.assets["background"], anchor="nw")
        # Draw player (with blinking effect)
        if self.env.blink_counter % 4 < 2:  # Blinking effect
            self.canvas.create_image(80, self.env.player_y, image=self.assets["player"], anchor="nw")
        # Draw obstacles
        for obs in self.env.obstacles:
            self.canvas.create_rectangle(
                obs["x"], 0,
                obs["x"] + OBSTACLE_WIDTH, obs["gap_y"],
                fill="blue", outline="white"
            )
            self.canvas.create_rectangle(
                obs["x"], obs["gap_y"] + GAP_SIZE,
                obs["x"] + OBSTACLE_WIDTH, HEIGHT,
                fill="blue", outline="white"
            )
        # Draw bullets
        for bullet in self.env.bullets:
            self.canvas.create_image(bullet["x"], bullet["y"], image=self.assets["bullet"], anchor="center")
        # Draw enemies
        for enemy in self.env.enemies:
            self.canvas.create_image(enemy["x"], enemy["y"], image=self.assets["enemy"], anchor="nw")

    def update(self):
        if not self.env.done:
            state = self.env._get_state()
            action = self.agent.choose_action(state)
            next_state, reward, done = self.env.step(action)
            self.agent.learn(state, action, reward, next_state)
            
            self.draw()
            self.score_label.config(text=f"Score: {self.env.score}")
            self.bullet_label.config(text=f"Missiles: {self.env.bullet_count}")
            self.life_label.config(text=f"Lives: {'♥'*self.env.lives}")
            self.epsilon_label.config(text=f"Explore: {self.agent.epsilon*100:.1f}%")
            self.root.after(20, self.update)
        else:
            self.canvas.create_text(WIDTH//2, HEIGHT//2, 
                                   text="GAME OVER!", 
                                   fill="red", font=("Impact", 48))
            self.root.after(1000, self._reset_game)

    def _reset_game(self):
        self.env.reset()
        self.canvas.delete("all")
        self.update()

# ========================
# Start the game
# ========================
if __name__ == "__main__":
    root = tk.Tk()
    root.title("AI Space Shooter Pro")
    GameWindow(root)
    root.mainloop()